In [28]:
import numpy as np
import random

In [29]:
def read_tsp(filename):
    coords = []
    with open(filename) as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3 and parts[0].isdigit():
                coords.append((float(parts[1]), float(parts[2])))
    return np.array(coords)

In [30]:
def calc_distance_matrix(coords):
    n = len(coords)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist[i, j] = np.linalg.norm(coords[i] - coords[j]) 
    return dist

In [31]:
def create_chromosome(n):
    chrom = list(range(n))
    random.shuffle(chrom)
    return chrom

def route_length(chrom, dist):
    return sum(dist[chrom[i], chrom[(i+1)%len(chrom)]] for i in range(len(chrom)))

def fitness(chrom, dist):
    return 1.0 / route_length(chrom, dist)


In [32]:
def tournament_selection(pop, fits, k=3):
    selected = random.sample(list(zip(pop, fits)), k)
    return max(selected, key=lambda x: x[1])[0][:]


In [33]:
def order_crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[a:b] = p1[a:b]
    fill = [c for c in p2 if c not in child]
    idx = 0
    for i in range(size):
        if child[i] is None:
            child[i] = fill[idx]
            idx += 1
    return child

In [34]:
def pmx_crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[a:b] = p1[a:b]
    for i in range(a, b):
        if p2[i] not in child:
            pos = i
            while True:
                val = p1[pos]
                pos = p2.index(val)
                if child[pos] is None:
                    child[pos] = p2[i]
                    break
    for i in range(size):
        if child[i] is None:
            child[i] = p2[i]
    return child

In [35]:
def cycle_crossover(p1, p2):
    size = len(p1)
    child = [None]*size
    idx = 0
    while None in child:
        if child[idx] is None:
            val = p1[idx]
            while True:
                child[idx] = p1[idx]
                idx = p1.index(p2[idx])
                if child[idx] is not None:
                    break
        idx = child.index(None) if None in child else 0
    return child

In [36]:
def swap_mutation(chrom):
    a, b = random.sample(range(len(chrom)), 2)
    chrom[a], chrom[b] = chrom[b], chrom[a]
    return chrom

def inversion_mutation(chrom):
    a, b = sorted(random.sample(range(len(chrom)), 2))
    chrom[a:b] = reversed(chrom[a:b])
    return chrom

def scramble_mutation(chrom):
    a, b = sorted(random.sample(range(len(chrom)), 2))
    subset = chrom[a:b]
    random.shuffle(subset)
    chrom[a:b] = subset
    return chrom

In [37]:
def genetic_algorithm_tsp(filename, pop_size=100, generations=500, crossover_rate=0.9, mutation_rate=0.2):
    coords = read_tsp(filename)
    dist = calc_distance_matrix(coords)
    n = len(coords)
    pop = [create_chromosome(n) for _ in range(pop_size)]
    best_chrom = None
    best_fit = -np.inf

    for gen in range(generations):
        fits = [fitness(chrom, dist) for chrom in pop]
        new_pop = []
        for _ in range(pop_size):
            p1 = tournament_selection(pop, fits)
            p2 = tournament_selection(pop, fits)
            if random.random() < crossover_rate:
                cross = random.choice([order_crossover, pmx_crossover, cycle_crossover])
                child = cross(p1, p2)
            else:
                child = p1[:]
            if random.random() < mutation_rate:
                mut = random.choice([swap_mutation, inversion_mutation, scramble_mutation])
                child = mut(child)
            new_pop.append(child)
        pop = new_pop
        
        gen_best_fit = max(fits)
        if gen_best_fit > best_fit:
            best_fit = gen_best_fit
            best_chrom = pop[np.argmax(fits)]
        if gen % 50 == 0:
            print(f"Generation {gen}: Best length = {1.0/best_fit:.2f}")
    print(f"Best route length: {route_length(best_chrom, dist):.2f}")
    return best_chrom, coords

In [38]:
best, coords = genetic_algorithm_tsp("TemplateRequirements/Template&Requirements/GA-TSP/city1.tsp", pop_size=200, generations=500)

Generation 0: Best length = 88337.59
Generation 50: Best length = 35938.50
Generation 100: Best length = 30411.52
Generation 150: Best length = 29811.31
Generation 200: Best length = 29596.28
Generation 250: Best length = 29258.34
Generation 300: Best length = 29258.34
Generation 350: Best length = 29258.34
Generation 400: Best length = 29258.34
Generation 450: Best length = 29258.34
Best route length: 29258.34


In [39]:
best, coords = genetic_algorithm_tsp("TemplateRequirements/Template&Requirements/GA-TSP/city2.tsp", pop_size=200, generations=500)

Generation 0: Best length = 22185.67
Generation 50: Best length = 10632.77
Generation 100: Best length = 7767.75
Generation 150: Best length = 6876.22
Generation 200: Best length = 6708.81
Generation 250: Best length = 6664.11
Generation 300: Best length = 6664.11
Generation 350: Best length = 6659.91
Generation 400: Best length = 6659.91
Generation 450: Best length = 6659.91
Best route length: 8237.36


In [40]:
best, coords = genetic_algorithm_tsp("TemplateRequirements/Template&Requirements/GA-TSP/city3.tsp", pop_size=200, generations=500)

Generation 0: Best length = 83869.61
Generation 50: Best length = 59190.38
Generation 100: Best length = 48143.32
Generation 150: Best length = 40721.42
Generation 200: Best length = 35373.63
Generation 250: Best length = 31753.58
Generation 300: Best length = 28804.46
Generation 350: Best length = 26862.80
Generation 400: Best length = 25240.53
Generation 450: Best length = 23043.44
Best route length: 21754.84
